In [28]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import pickle as pkl
import random
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
import tensorflow as tf
import keras
import keras_metrics as km
from keras import initializers

from keras.callbacks import TensorBoard
from keras import regularizers
from custom_callbacks import LrFinder
from custom_callbacks import CycleLearner
from custom_callbacks import reset_weights
from keras.callbacks import ModelCheckpoint
from time import time
from keras.layers import Input, Dense,Dropout,BatchNormalization,LSTM,GRU,Bidirectional,Conv2D, MaxPool2D, Flatten, GlobalAvgPool2D, GlobalMaxPool2D,merge,CuDNNGRU,CuDNNLSTM
from keras.models import Model,Sequential
from sklearn.model_selection import train_test_split
from keras.optimizers import SGD
from keras.optimizers import Adam
from keras.optimizers import Adagrad
from keras.optimizers import Adamax
from keras.optimizers import Nadam
from keras.optimizers import RMSprop
from keras import backend as K
import math
import json
import keras.backend as K
from keras.layers import Lambda

# Get the dictionary containing each metric and the loss for each epoch

train_filepath = "train_workspace/"
animalList = ["Human","Pig","Chicken","Rat","Mouse","Dog"]
levelList = ["Strict","Relaxed","Intermediate"]
dataList = ["Ohnologs","No-Ohnologs","Paralogs"]
dataNameList = ["ohnologs","no-ohnologs","paralog"]
sns.set_style("darkgrid")


In [2]:
working_level = "Strict"
kmer_chosen = 8
type_chosen = "cdna"

dataset_complete = pd.read_pickle(train_filepath + working_level + "/datasets/2_paralog_dataset_complete-" + str(kmer_chosen) + "-" + type_chosen + ".pkl")
dataset_complete_soft = pd.read_pickle(train_filepath + working_level + "/datasets/3_paralog_dataset_complete-" + str(kmer_chosen) + "-" + type_chosen + ".pkl")

In [3]:
new_classes = []
for index, row in dataset_complete_soft.iterrows():
    if(row["Is_Ohnolog"] == 1):
        new_classes.append(1)
    else:
        if(row["Is_Paralog"] == 0):
            new_classes.append(0)
        else:
            new_classes.append(2)
dataset_complete_soft["Dup_Class"] = new_classes
dataset_complete_soft.head()

,Specie_Chicken,Specie_Dog,Specie_Human,Specie_Mouse,Specie_Pig,Specie_Rat,Is_Ohnolog,Is_Paralog,Sequence-1 GC,Sequence-1 Length,...,Embedding2_91,Embedding2_92,Embedding2_93,Embedding2_94,Embedding2_95,Embedding2_96,Embedding2_97,Embedding2_98,Embedding2_99,Dup_Class
0,0,0,1,0,0,0,1,0.0,0.429090,0.030467,...,35.407509,23.369914,8.144324,-26.702099,241.207709,-12.695194,-79.573883,121.379307,-69.633950,1
1,0,0,1,0,0,0,1,0.0,0.441268,0.053554,...,12.556002,23.217834,3.162621,-18.980361,204.830682,-20.962017,-61.163222,122.543777,-26.694989,1
2,0,0,1,0,0,0,1,0.0,0.357972,0.111632,...,-6.850837,12.314794,-3.387437,-5.403946,93.521862,4.187927,-35.880841,50.565405,-9.853733,1
3,0,0,1,0,0,0,1,0.0,0.684226,0.031676,...,68.287420,6.295047,37.820265,-33.841090,230.696114,-60.500167,-82.975793,174.224105,-111.569089,1
4,0,0,1,0,0,0,1,0.0,0.402629,0.018732,...,18.025175,13.675959,21.304353,-2.702105,162.972440,-8.209692,-46.736790,62.650600,-39.056701,1


In [4]:
new_classes = []
for index, row in dataset_complete.iterrows():
    if(row["Is_Ohnolog"] == 1):
        new_classes.append(1)
    else:
        if(row["Is_Paralog"] == 0):
            new_classes.append(0)
        else:
            new_classes.append(2)
dataset_complete["Dup_Class"] = new_classes
dataset_complete.head()

,Specie_Chicken,Specie_Dog,Specie_Human,Specie_Mouse,Specie_Pig,Specie_Rat,Is_Ohnolog,Is_Paralog,Sequence-1 GC,Sequence-1 Length,...,Embedding2_91,Embedding2_92,Embedding2_93,Embedding2_94,Embedding2_95,Embedding2_96,Embedding2_97,Embedding2_98,Embedding2_99,Dup_Class
0,0,0,1,0,0,0,1,0.0,0.429090,0.115183,...,35.407509,23.369914,8.144324,-26.702099,241.207709,-12.695194,-79.573883,121.379307,-69.633950,1
1,0,0,1,0,0,0,1,0.0,0.441268,0.202706,...,12.556002,23.217834,3.162621,-18.980361,204.830682,-20.962017,-61.163222,122.543777,-26.694989,1
2,0,0,1,0,0,0,1,0.0,0.357972,0.422880,...,-6.850837,12.314794,-3.387437,-5.403946,93.521862,4.187927,-35.880841,50.565405,-9.853733,1
3,0,0,1,0,0,0,1,0.0,0.684226,0.119763,...,68.287420,6.295047,37.820265,-33.841090,230.696114,-60.500167,-82.975793,174.224105,-111.569089,1
4,0,0,1,0,0,0,1,0.0,0.402629,0.070693,...,18.025175,13.675959,21.304353,-2.702105,162.972440,-8.209692,-46.736790,62.650600,-39.056701,1


In [5]:
blast_columns = ["Percent Identical Matches","Aligment Length","Nr Mismatch","Nr Gap Open","Evalue","Bit Score"]

e_values_columns = ["Evalue_Total","Evalue_High","Evalue_Medium","Evalue_Low"]

blast_types = ["High","Medium","Low","Total"]
blast_col_Total = [i + "_Total" for i in blast_columns] 
blast_col_High = [i + "_High" for i in blast_columns]
blast_col_Medium = [i + "_Medium" for i in blast_columns]
blast_col_Low = [i + "_Low" for i in blast_columns]

blast_cols_levels = blast_col_High + blast_col_Medium + blast_col_Low
total_columns = (blast_col_High + blast_col_Medium + blast_col_Low + blast_col_Total + ["Nr Hits"])

non_training_meta_features = ["Sequence-1","Sequence-2","Sequence-1 Id","Sequence-1-Transcript Id","Sequence-2 Id","Sequence-2-Transcript Id","Sequence-1-Transcript-Version","Sequence-2-Transcript-Version"]

sequence_1_metadata = ["Sequence-1 GC","Sequence-1 Length","Sequence-1-Chromosome","Seq-1-Biotype_protein_coding"]
sequence_2_metadata = ["Sequence-2 GC","Sequence-2 Length","Sequence-2-Chromosome","Seq-2-Biotype_protein_coding"]

sequence_1_metadata_diferential = ["Sequence-1 GC","Sequence-1 Length"]
sequence_2_metadata_diferential = ["Sequence-2 GC","Sequence-2 Length"]

sequence_1_metadata_categorical = ["Sequence-1-Chromosome","Seq-1-Biotype_protein_coding"]
sequence_2_metadata_categorical = ["Sequence-2-Chromosome","Seq-2-Biotype_protein_coding"]

species_metadata = ["Specie_Chicken","Specie_Dog","Specie_Human","Specie_Mouse","Specie_Pig","Specie_Rat"]

emb_size = 100
if(type_chosen == "cdna2"):
    emb_size = 200
embedding_1_cols = ["Embedding1_" + str(i) for i in range(0,emb_size)]
embedding_2_cols = ["Embedding2_" + str(i) for i in range(0,emb_size)]

In [6]:
def get_train_val_test(df):
    df.sample(frac=1,random_state=7)
    df_train, df_test = train_test_split(df, test_size=0.2,random_state=9,stratify=df["Is_Ohnolog"])
    df_train, df_val = train_test_split(df_train, test_size=0.2,random_state=3,stratify=df_train["Is_Ohnolog"])
    return (df_train,df_val,df_test)

In [7]:
def get_train_val_test_soft(df):
    df.sample(frac=1,random_state=7)
    df_train, df_test = train_test_split(df, test_size=0.2,random_state=9,stratify=df["Dup_Class"])
    df_train, df_val = train_test_split(df_train, test_size=0.2,random_state=3,stratify=df_train["Dup_Class"])
    return (df_train,df_val,df_test)

In [8]:
def save_log(log,name):
    log_dict = log.history    
    json.dump(log_dict, open(train_filepath + working_level + "/model_run_history/" + name + ".json", 'w'))

In [9]:
def get_log(name):
     return json.load(open(train_filepath + working_level + "/model_run_history/" + name + ".json", 'r'))

In [10]:
class EpochLogger(keras.callbacks.Callback):
    def __init__(self, display):
        self.step = 0
        self.display = display
        self.metric_cache = {}        

    def on_epoch_end(self, epoch, logs={}):
        self.step += 1
        for k in self.params['metrics']:
            if k in logs:
                self.metric_cache[k] = self.metric_cache.get(k, 0) + logs[k]
        if self.step % self.display == 0:
            metrics_log_default = ''            
            metrics_log_main = ''            
            metrics_log_aux = ''            
            for (k, v) in self.metric_cache.items():
                val = v / self.display             
                if("main" in k):
                    if abs(val) > 1e-3:
                        metrics_log_main += ' - %s: %.4f' % (k, val)
                        continue      
                    else:
                        metrics_log_main += ' - %s: %.4e' % (k, val)
                        continue                    
                        
                if("aux" in k): 
                    if abs(val) > 1e-3:
                        metrics_log_aux += ' - %s: %.4f' % (k, val)
                        continue      
                    else:
                        metrics_log_aux += ' - %s: %.4e' % (k, val)
                        continue      
                        
                if abs(val) > 1e-3:
                        metrics_log_default += ' - %s: %.4f' % (k, val)
                else:
                        metrics_log_default += ' - %s: %.4e' % (k, val)                        
                    
            print('step: {}/{} ... {}'.format(self.step,
                                          self.params['epochs'],
                                          metrics_log_default + "\n" + metrics_log_main + "\n" + metrics_log_aux))
            self.metric_cache.clear()

## Levels

In [11]:
def show_results_level(model, log,x_test,x_emb_test,y_test,bs,metrics_used):
    metrics_names,metrics = (model.metrics_names,model.evaluate([x_emb_test.values,x_test.values],[y_test.values,y_test.values], batch_size=bs,verbose=False))
    count = 0
    for i in range(0,len(metrics)):
        print(f'{metrics_names[i]}     = {metrics[i]:.4f}')                
    nr_metrics = len(metrics_used)
    fig, axes = plt.subplots(2, nr_metrics, figsize=(18,8))   
    
    i = 0
    for metric in metrics_used:
        match_metrics = [model_metric.split('_')[0] for model_metric in metrics_names if metric in model_metric]
        print(match_metrics)
        if(metric == "loss"):
            match_metrics.remove("loss")
        for match in match_metrics:        
            if(i>=nr_metrics):
                axes[1][i-nr_metrics].plot(log[match + "_" + metric], label="Train")            
                axes[1][i-nr_metrics].plot(log["val_" + match + "_" + metric], label="Val")       
                axes[1][i-nr_metrics].set_xlabel('epoch'); 
                axes[1][i-nr_metrics].set_ylabel(metric + "_" + match)                            
            else:
                axes[0][i].plot(log[match + "_" + metric], label="Train")            
                axes[0][i].plot(log["val_" + match + "_" + metric], label="Val")       
                axes[0][i].set_xlabel('epoch'); 
                axes[0][i].set_ylabel(metric + "_" + match)                            
                
            i += 1            
    for ax in axes:
        for a in ax:
             a.legend()

In [12]:
def show_results_level_RNN(model, log,x_test,x_emb_test,y_test,bs,metrics_used):
    metrics_names,metrics = (model.metrics_names,model.evaluate([x_emb_test.values.reshape(x_emb_test.shape[0],x_emb_test.shape[1],1),x_test.values],[y_test.values,y_test.values], batch_size=bs,verbose=False))
    count = 0
    for i in range(0,len(metrics)):
        print(f'{metrics_names[i]}     = {metrics[i]:.4f}')                
    nr_metrics = len(metrics_used)
    fig, axes = plt.subplots(2, nr_metrics, figsize=(18,8))   
    
    i = 0
    for metric in metrics_used:
        match_metrics = [model_metric.split('_')[0] for model_metric in metrics_names if metric in model_metric]
        print(match_metrics)
        if(metric == "loss"):
            match_metrics.remove("loss")
        for match in match_metrics:        
            if(i>=nr_metrics):
                axes[1][i-nr_metrics].plot(log[match + "_" + metric], label="Train")            
                axes[1][i-nr_metrics].plot(log["val_" + match + "_" + metric], label="Val")       
                axes[1][i-nr_metrics].set_xlabel('epoch'); 
                axes[1][i-nr_metrics].set_ylabel(metric + "_" + match)                            
            else:
                axes[0][i].plot(log[match + "_" + metric], label="Train")            
                axes[0][i].plot(log["val_" + match + "_" + metric], label="Val")       
                axes[0][i].set_xlabel('epoch'); 
                axes[0][i].set_ylabel(metric + "_" + match)                            
                
            i += 1            
    for ax in axes:
        for a in ax:
             a.legend()

In [13]:
def level_model(df_meta_input,df_embeddings,name):    
    input_embedding = Input(shape=(len(df_embeddings.columns),), name='embedding_input')        
    emb_x = Dense(512, activation='relu',use_bias=False,kernel_regularizer=regularizers.l2(0.01))(input_embedding)    
    emb_x = BatchNormalization()(emb_x)
    emb_x = Dropout(0.4)(emb_x)
    emb_x = Dense(512, activation='relu',kernel_regularizer=regularizers.l2(0.01))(emb_x)
    emb_x = Dropout(0.4)(emb_x)
    emb_x = Dense(256, activation='relu',use_bias = False,kernel_regularizer=regularizers.l2(0.01))(emb_x)    
    emb_x = Dropout(0.3)(emb_x)
    emb_x = BatchNormalization()(emb_x)
    emb_x = Dense(128, activation='relu',kernel_regularizer=regularizers.l2(0.01))(emb_x)
    emb_x = Dropout(0.3)(emb_x)
    emb_x = Dense(64, activation='relu',kernel_regularizer=regularizers.l2(0.01))(emb_x)    
    emb_x = Dropout(0.3)(emb_x)
    emb_x = Dense(32, activation='relu',use_bias = False)(emb_x)
    emb_x = BatchNormalization()(emb_x)
    emb_x = Dense(16, activation='relu',kernel_regularizer=regularizers.l2(0.01))(emb_x)        
    emb_x_out = Dense(1, activation='sigmoid',name="aux")(emb_x)
    
    
    meta_input = Input(shape=(len(df_meta_input.columns),), name='meta_input')
    x = keras.layers.concatenate([emb_x, meta_input])
    x = Dense(128, activation='relu',use_bias=False)(x)    
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    x = Dense(64, activation='relu',use_bias=False,kernel_regularizer=regularizers.l2(0.001))(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    x = Dense(32, activation='relu',use_bias=False,kernel_regularizer=regularizers.l2(0.001))(x)   
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    x = Dense(32, activation='relu',use_bias=False,kernel_regularizer=regularizers.l2(0.001))(x)    
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    x = Dense(16, activation='relu',kernel_regularizer=regularizers.l2(0.0005))(x)    
    x = Dropout(0.1)(x)
    x = Dense(8, activation='relu',use_bias=False,kernel_regularizer=regularizers.l2(0.0005))(x)
    x = BatchNormalization()(x)
    predictions = Dense(1, activation='sigmoid',name="main")(x)

    # This creates a model that includes
    # the Input layer and three Dense layers                       
    model_created = Model(inputs=[input_embedding, meta_input], outputs=[emb_x_out,predictions])
    model_created.Name = name
    return model_created

In [14]:
def level_model_soft(df_meta_input,df_embeddings,name):    
    input_embedding = Input(shape=(len(df_embeddings.columns),), name='embedding_input')        
    emb_x = Dense(512, activation='relu',use_bias=False,kernel_regularizer=regularizers.l2(0.01))(input_embedding)    
    emb_x = BatchNormalization()(emb_x)
    emb_x = Dropout(0.4)(emb_x)
    emb_x = Dense(512, activation='relu',kernel_regularizer=regularizers.l2(0.01))(emb_x)
    emb_x = Dropout(0.4)(emb_x)
    emb_x = Dense(256, activation='relu',use_bias = False,kernel_regularizer=regularizers.l2(0.01))(emb_x)    
    emb_x = Dropout(0.3)(emb_x)
    emb_x = BatchNormalization()(emb_x)
    emb_x = Dense(128, activation='relu',kernel_regularizer=regularizers.l2(0.01))(emb_x)
    emb_x = Dropout(0.3)(emb_x)
    emb_x = Dense(64, activation='relu',kernel_regularizer=regularizers.l2(0.01))(emb_x)    
    emb_x = Dropout(0.3)(emb_x)
    emb_x = Dense(32, activation='relu',use_bias = False)(emb_x)
    emb_x = BatchNormalization()(emb_x)
    emb_x = Dense(16, activation='relu',kernel_regularizer=regularizers.l2(0.01))(emb_x)        
    emb_x_out = Dense(3, activation='softmax',name="aux")(emb_x)
    
    
    meta_input = Input(shape=(len(df_meta_input.columns),), name='meta_input')
    x = keras.layers.concatenate([emb_x, meta_input])
    x = Dense(128, activation='relu',use_bias=False)(x)    
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    x = Dense(64, activation='relu',use_bias=False,kernel_regularizer=regularizers.l2(0.001))(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    x = Dense(32, activation='relu',use_bias=False,kernel_regularizer=regularizers.l2(0.001))(x)   
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    x = Dense(32, activation='relu',use_bias=False,kernel_regularizer=regularizers.l2(0.001))(x)    
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    x = Dense(16, activation='relu',kernel_regularizer=regularizers.l2(0.0005))(x)    
    x = Dropout(0.1)(x)
    x = Dense(8, activation='relu',use_bias=False,kernel_regularizer=regularizers.l2(0.0005))(x)
    x = BatchNormalization()(x)
    predictions = Dense(3, activation='softmax',name="main")(x)

    # This creates a model that includes
    # the Input layer and three Dense layers                       
    model_created = Model(inputs=[input_embedding, meta_input], outputs=[emb_x_out,predictions])
    model_created.Name = name
    return model_created


In [15]:
def fit_model_level(train_x,train_emb_x,train_y,val_x,val_emb_x,val_y,model_train,n_epochs,optimizer,batchsize,loss_weigths,verb):
    tensorboard = TensorBoard(log_dir=train_filepath + working_level + "/board_logs/" + model_train.Name + "-" + "-{}".format(time()))
    checkpoint = ModelCheckpoint(train_filepath + working_level + "/model_checkpoints/{0}-check-{{epoch:02d}}-{{val_main_acc:.2f}}.hdf5".format(model_train.Name),save_weights_only=True, period = int(n_epochs/5))
    best_model_save = ModelCheckpoint(train_filepath + working_level + "/model_checkpoints/{0}-best.hdf5".format(model_train.Name), monitor='val_main_acc',save_weights_only=True,  save_best_only=True, mode='max')
    logger = EpochLogger(display=50)

    model_train.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy',km.binary_recall()],loss_weights=loss_weigths)
    size = len(train_emb_x)
    return model_train.fit([train_emb_x,train_x], y= [train_y,train_y],verbose = verb,validation_data=([val_emb_x,val_x],[val_y,val_y]),epochs = n_epochs,batch_size=batchsize,callbacks = [tensorboard,checkpoint,best_model_save,logger])  # starts training

In [16]:
def fit_model_level_soft(train_x,train_emb_x,train_y,val_x,val_emb_x,val_y,model_train,n_epochs,optimizer,batchsize,loss_weigths,verb):
    tensorboard = TensorBoard(log_dir=train_filepath + working_level + "/board_logs/" + model_train.Name + "-" + "-{}".format(time()))
    checkpoint = ModelCheckpoint(train_filepath + working_level + "/model_checkpoints/{0}-check-{{epoch:02d}}-{{val_main_acc:.2f}}.hdf5".format(model_train.Name),save_weights_only=True, period = int(n_epochs/5))
    best_model_save = ModelCheckpoint(train_filepath + working_level + "/model_checkpoints/{0}-best.hdf5".format(model_train.Name), monitor='val_main_acc',save_weights_only=True,  save_best_only=True, mode='max')
    logger = EpochLogger(display=50)

    model_train.compile(optimizer=optimizer,loss='sparse_categorical_crossentropy',metrics=['accuracy',km.sparse_categorical_recall()],loss_weights=loss_weigths)
    size = len(train_emb_x)
    return model_train.fit([train_emb_x,train_x], y= [train_y,train_y],verbose = verb,validation_data=([val_emb_x,val_x],[val_y,val_y]),epochs = n_epochs,batch_size=batchsize,callbacks = [tensorboard,checkpoint,best_model_save,logger])  # starts training

In [17]:
species_dict = {}
for specie in species_metadata:
    if(specie in ["Specie_Chicken","Specie_Dog","Specie_Human","Specie_Mouse","Specie_Pig"]):
        continue
        
    dataset_complete_no_species = dataset_complete[dataset_complete[specie] == 0]    
    dataset_complete_species = dataset_complete[dataset_complete[specie] == 1]    
    
    df_train,df_val,df_test = get_train_val_test(dataset_complete_no_species)
    
    df_train_x = df_train.drop(["Is_Ohnolog","Is_Paralog","Dup_Class"],axis=1)
    df_train_x = df_train_x.drop(embedding_1_cols,axis=1)
    df_train_x = df_train_x.drop(embedding_2_cols,axis=1)

    df_train_emb_x = pd.concat([df_train[embedding_1_cols],df_train[embedding_2_cols]],axis=1,sort=False)
    df_train_y = df_train["Is_Ohnolog"]

    df_val_x = df_val.drop(["Is_Ohnolog","Is_Paralog","Dup_Class"],axis=1)
    df_val_x = df_val_x.drop(embedding_1_cols,axis=1)
    df_val_x = df_val_x.drop(embedding_2_cols,axis=1)

    df_val_emb_x = pd.concat([df_val[embedding_1_cols],df_val[embedding_2_cols]],axis=1,sort=False)
    df_val_y = df_val["Is_Ohnolog"]

    df_test_x = df_test.drop(["Is_Ohnolog","Is_Paralog","Dup_Class"],axis=1)
    df_test_x = df_test_x.drop(embedding_1_cols,axis=1)
    df_test_x = df_test_x.drop(embedding_2_cols,axis=1)

    df_test_emb_x =  pd.concat([df_test[embedding_1_cols],df_test[embedding_2_cols]],axis=1,sort=False)
    df_test_y = df_test["Is_Ohnolog"]
    
    model = level_model(df_train_x,df_train_emb_x,"Level_model " + specie)
    
    log = fit_model_level(df_train_x,df_train_emb_x,df_train_y,df_val_x,df_val_emb_x,df_val_y,model,600,Adamax(),256,[0.2,1],0)
    
    df_test_species_x = dataset_complete_species.drop(["Is_Ohnolog","Is_Paralog","Dup_Class"],axis=1)
    df_test_species_x = df_test_species_x.drop(embedding_1_cols,axis=1)
    df_test_species_x = df_test_species_x.drop(embedding_2_cols,axis=1)

    df_test_species_emb_x =  pd.concat([dataset_complete_species[embedding_1_cols],dataset_complete_species[embedding_2_cols]],axis=1,sort=False)
    df_test_species_y = dataset_complete_species["Is_Ohnolog"]
    
    species_dict[specie] = {}
    
    print("Evaluando especie Test: " + specie)
    metrics_names,metrics = (model.metrics_names,model.evaluate([df_test_emb_x.values,df_test_x.values],[df_test_y.values,df_test_y.values], batch_size=256,verbose=False))
    count = 0
    for i in range(0,len(metrics)):
        print(f'{metrics_names[i]}     = {metrics[i]:.4f}')  
    print("\n")
    
    species_dict[specie]["Test"] = metrics[5]
            
        
    print("Evaluando especie sin balancear: " + specie)
    metrics_names,metrics = (model.metrics_names,model.evaluate([df_test_species_emb_x.values,df_test_species_x.values],[df_test_species_y.values,df_test_species_y.values], batch_size=256,verbose=False))
    count = 0
    for i in range(0,len(metrics)):
        print(f'{metrics_names[i]}     = {metrics[i]:.4f}')  
    print("\n")
    
    species_dict[specie]["Sin Balanceo"] = metrics[5]
    
    df_species_ohnologs = dataset_complete_species[dataset_complete_species["Is_Ohnolog"] == 1]
    df_species_no_ohnologs = dataset_complete_species[dataset_complete_species["Is_Ohnolog"] == 0]
    
    if(len(df_species_ohnologs)>len(df_species_no_ohnologs)):
        df_species_new = df_species_no_ohnologs.append(df_species_ohnologs.sample(len(df_species_no_ohnologs)))        
    else:
        df_species_new = df_species_ohnologs.append(df_species_no_ohnologs.sample(len(df_species_ohnologs)))
                 
    if(len(df_species_new) == 0):
        df_species_new = df_species_ohnologs
        
    df_test_species_x = df_species_new.drop(["Is_Ohnolog","Is_Paralog","Dup_Class"],axis=1)
    df_test_species_x = df_test_species_x.drop(embedding_1_cols,axis=1)
    df_test_species_x = df_test_species_x.drop(embedding_2_cols,axis=1)

    df_test_species_emb_x =  pd.concat([df_species_new[embedding_1_cols],df_species_new[embedding_2_cols]],axis=1,sort=False)
    df_test_species_y = df_species_new["Is_Ohnolog"]    
    
    print("Evaluando especie balanceada: " + specie)
    metrics_names,metrics = (model.metrics_names,model.evaluate([df_test_species_emb_x.values,df_test_species_x.values],[df_test_species_y.values,df_test_species_y.values], batch_size=256,verbose=False))
    count = 0
    for i in range(0,len(metrics)):
        print(f'{metrics_names[i]}     = {metrics[i]:.4f}')  
    print("\n")
    
    species_dict[specie]["Balanceadas"] = metrics[5]            

step: 50/600 ...  - loss: 1.5250 - val_loss: 1.3687
 - main_loss: 0.3903 - main_acc: 0.8202 - main_recall: 0.6250 - val_main_loss: 0.3704 - val_main_acc: 0.8363 - val_main_recall: 0.6594
 - aux_loss: 0.5159 - aux_acc: 0.7143 - aux_recall: 0.6225 - val_aux_loss: 0.5042 - val_aux_acc: 0.7390 - val_aux_recall: 0.6435
step: 100/600 ...  - loss: 0.4519 - val_loss: 0.4422
 - main_loss: 0.3249 - main_acc: 0.8589 - main_recall: 0.6438 - val_main_loss: 0.3203 - val_main_acc: 0.8578 - val_main_recall: 0.6701
 - aux_loss: 0.4986 - aux_acc: 0.7226 - aux_recall: 0.6408 - val_aux_loss: 0.4738 - val_aux_acc: 0.7437 - val_aux_recall: 0.6537
step: 150/600 ...  - loss: 0.4240 - val_loss: 0.4215
 - main_loss: 0.3054 - main_acc: 0.8680 - main_recall: 0.6502 - val_main_loss: 0.3075 - val_main_acc: 0.8628 - val_main_recall: 0.6749
 - aux_loss: 0.4958 - aux_acc: 0.7239 - aux_recall: 0.6471 - val_aux_loss: 0.4728 - val_aux_acc: 0.7450 - val_aux_recall: 0.6583
step: 200/600 ...  - loss: 0.4124 - val_loss: 0.41

In [33]:
species_dict

{'Specie_Rat': {'Test': 0.8733780763293273,
  'Sin Balanceo': 0.7416687553402656,
  'Balanceadas': 0.7435828884017659}}

In [20]:
species_cross_dict = {}
for specie in species_metadata:
    dataset_complete_no_species = dataset_complete[dataset_complete[specie] == 0]    
    dataset_complete_species = dataset_complete[dataset_complete[specie] == 1]    
        
    df_no_species = dataset_complete_no_species[dataset_complete_no_species[specie] == 0]
    df_train, df_val = train_test_split(df_no_species, test_size=0.2,random_state=9,stratify=df_no_species["Is_Ohnolog"])
    df_train_x = df_train.drop(["Is_Ohnolog","Is_Paralog","Dup_Class"],axis=1)
    df_train_x = df_train_x.drop(embedding_1_cols,axis=1)
    df_train_x = df_train_x.drop(embedding_2_cols,axis=1)
    df_train_emb_x = pd.concat([df_train[embedding_1_cols],df_train[embedding_2_cols]],axis=1,sort=False)

    model = level_model(df_train_x,df_train_emb_x,"Level_model_cross" + specie  )
    specie_metadata_copy = species_metadata[:]
    specie_metadata_copy.remove(specie)

    for specie_cross in specie_metadata_copy:
        df_no_species = dataset_complete_no_species[dataset_complete_no_species[specie_cross] == 0]
        df_train, df_val = train_test_split(df_no_species, test_size=0.2,random_state=9,stratify=df_no_species["Is_Ohnolog"])

        df_train_x = df_train.drop(["Is_Ohnolog","Is_Paralog","Dup_Class"],axis=1)
        df_train_x = df_train_x.drop(embedding_1_cols,axis=1)
        df_train_x = df_train_x.drop(embedding_2_cols,axis=1)

        df_train_emb_x = pd.concat([df_train[embedding_1_cols],df_train[embedding_2_cols]],axis=1,sort=False)
        df_train_y = df_train["Is_Ohnolog"]

        df_val_x = df_val.drop(["Is_Ohnolog","Is_Paralog","Dup_Class"],axis=1)
        df_val_x = df_val_x.drop(embedding_1_cols,axis=1)
        df_val_x = df_val_x.drop(embedding_2_cols,axis=1)

        df_val_emb_x = pd.concat([df_val[embedding_1_cols],df_val[embedding_2_cols]],axis=1,sort=False)
        df_val_y = df_val["Is_Ohnolog"]

        print("Entrenando sin especie: " + specie_cross)
        log = fit_model_level(df_train_x,df_train_emb_x,df_train_y,df_val_x,df_val_emb_x,df_val_y,model,150,Adamax(),256,[0.2,1],0)
        
    
    df_test_species_x = dataset_complete_species.drop(["Is_Ohnolog","Is_Paralog","Dup_Class"],axis=1)
    df_test_species_x = df_test_species_x.drop(embedding_1_cols,axis=1)
    df_test_species_x = df_test_species_x.drop(embedding_2_cols,axis=1)

    df_test_species_emb_x =  pd.concat([dataset_complete_species[embedding_1_cols],dataset_complete_species[embedding_2_cols]],axis=1,sort=False)
    df_test_species_y = dataset_complete_species["Is_Ohnolog"]
    
    species_cross_dict[specie] = {}
    
    print("Evaluando especie sin balancear: " + specie)
    metrics_names,metrics = (model.metrics_names,model.evaluate([df_test_species_emb_x.values,df_test_species_x.values],[df_test_species_y.values,df_test_species_y.values], batch_size=256,verbose=False))
    count = 0
    for i in range(0,len(metrics)):
        print(f'{metrics_names[i]}     = {metrics[i]:.4f}')  
    print("\n")
    
    species_cross_dict[specie]["Sin Balanceo"] = metrics[5]
    
    df_species_ohnologs = dataset_complete_species[dataset_complete_species["Is_Ohnolog"] == 1]
    df_species_no_ohnologs = dataset_complete_species[dataset_complete_species["Is_Ohnolog"] == 0]
    
    if(len(df_species_ohnologs)>len(df_species_no_ohnologs)):
        df_species_new = df_species_no_ohnologs.append(df_species_ohnologs.sample(len(df_species_no_ohnologs)))        
    else:
        df_species_new = df_species_ohnologs.append(df_species_no_ohnologs.sample(len(df_species_ohnologs)))
        
        
    if(len(df_species_new) == 0):
        df_species_new = df_species_ohnologs
        
    df_test_species_x = df_species_new.drop(["Is_Ohnolog","Is_Paralog","Dup_Class"],axis=1)
    df_test_species_x = df_test_species_x.drop(embedding_1_cols,axis=1)
    df_test_species_x = df_test_species_x.drop(embedding_2_cols,axis=1)

    df_test_species_emb_x =  pd.concat([df_species_new[embedding_1_cols],df_species_new[embedding_2_cols]],axis=1,sort=False)
    df_test_species_y = df_species_new["Is_Ohnolog"]                    
    
    print("Evaluando especie balanceada: " + specie)
    metrics_names,metrics = (model.metrics_names,model.evaluate([df_test_species_emb_x.values,df_test_species_x.values],[df_test_species_y.values,df_test_species_y.values], batch_size=256,verbose=False))
    count = 0
    for i in range(0,len(metrics)):
        print(f'{metrics_names[i]}     = {metrics[i]:.4f}')  
    print("\n")
    
    species_cross_dict[specie]["Balanceadas"] = metrics[5]      

Entrenando sin especie: Specie_Dog
step: 50/150 ...  - loss: 1.9420 - val_loss: 1.7840
 - main_loss: 0.4532 - main_acc: 0.7884 - main_recall: 0.5635 - val_main_loss: 0.4332 - val_main_acc: 0.8038 - val_main_recall: 0.5704
 - aux_loss: 0.5818 - aux_acc: 0.6635 - aux_recall: 0.5613 - val_aux_loss: 0.5780 - val_aux_acc: 0.6690 - val_aux_recall: 0.5645
step: 100/150 ...  - loss: 0.5249 - val_loss: 0.5220
 - main_loss: 0.3823 - main_acc: 0.8320 - main_recall: 0.6063 - val_main_loss: 0.3803 - val_main_acc: 0.8348 - val_main_recall: 0.6305
 - aux_loss: 0.5697 - aux_acc: 0.6688 - aux_recall: 0.6036 - val_aux_loss: 0.5658 - val_aux_acc: 0.6779 - val_aux_recall: 0.6248
step: 150/150 ...  - loss: 0.4962 - val_loss: 0.4931
 - main_loss: 0.3631 - main_acc: 0.8409 - main_recall: 0.6105 - val_main_loss: 0.3608 - val_main_acc: 0.8426 - val_main_recall: 0.6471
 - aux_loss: 0.5671 - aux_acc: 0.6696 - aux_recall: 0.6079 - val_aux_loss: 0.5633 - val_aux_acc: 0.6772 - val_aux_recall: 0.6422
Entrenando sin 

In [32]:
species_cross_dict

{'Specie_Chicken': {'Sin Balanceo': 0.8183273321249429,
  'Balanceadas': 0.8295318115086687},
 'Specie_Dog': {'Sin Balanceo': 0.7151090978422789,
  'Balanceadas': 0.7151090978422789},
 'Specie_Human': {'Sin Balanceo': 0.7561962573770689,
  'Balanceadas': 0.7606221555035465},
 'Specie_Mouse': {'Sin Balanceo': 0.8072699741042504,
  'Balanceadas': 0.8072699741042504},
 'Specie_Pig': {'Sin Balanceo': 0.7972048334479072,
  'Balanceadas': 0.7712235953191117},
 'Specie_Rat': {'Sin Balanceo': 0.6898020546975754,
  'Balanceadas': 0.6997326202571074}}

## Siames

In [26]:
def get_tensor_embedding_layer(input_embedding):    
    emb_x = Dense(512, activation='elu',use_bias=False,kernel_regularizer=regularizers.l2(0.001))(input_embedding)       
    emb_x = BatchNormalization()(emb_x)
    emb_x = Dropout(0.4)(emb_x)    
    emb_x = Dense(256,use_bias = False, activation='elu',kernel_regularizer=regularizers.l1(0.0005))(emb_x)            
    emb_x = BatchNormalization()(emb_x)    
    emb_x = Dropout(0.4)(emb_x)    
    emb_x = Dense(128,use_bias = False, activation='elu',kernel_regularizer=regularizers.l1(0.0005))(emb_x)            
    emb_x = BatchNormalization()(emb_x)    
    emb_x = Dense(64, use_bias = False, activation='elu',kernel_regularizer=regularizers.l1(0.0005))(emb_x)            
    emb_x = BatchNormalization()(emb_x)    
    emb_x = Dense(32, use_bias = False, activation='elu',kernel_regularizer=regularizers.l1(0.0005))(emb_x)                
    emb_x = BatchNormalization()(emb_x)    
    return emb_x


In [21]:
def level_siames_merge_layer(df_meta_input,df_embeddings_1,df_embeddings_2,name,bs):    
    
    input_embedding_1 = Input(shape=(len(df_embeddings_1.columns),), name='embedding_input_1')        
    input_embedding_2 = Input(shape=(len(df_embeddings_2.columns),), name='embedding_input_2')        
    
    emb_x_1 = get_tensor_embedding_layer(input_embedding_1)    
    emb_x_2 = get_tensor_embedding_layer(input_embedding_2)    
    
    L1_layer = Lambda(lambda tensors:K.abs(tensors[0] - tensors[1]))
    L1_distance = L1_layer([emb_x_1, emb_x_2])
    
    emb_x_out = Dense(1, activation='sigmoid',name="aux")(L1_distance)                
    
    meta_input = Input(shape=(len(df_meta_input.columns),), name='meta_input')    
    
    x = keras.layers.concatenate([L1_distance,meta_input])
    x = Dense(128, activation='relu',use_bias=False)(x)    
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    x = Dense(64, activation='relu',use_bias=False,kernel_regularizer=regularizers.l2(0.001))(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    x = Dense(32, activation='relu',use_bias=False,kernel_regularizer=regularizers.l2(0.001))(x)   
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    x = Dense(32, activation='relu',use_bias=False,kernel_regularizer=regularizers.l2(0.001))(x)    
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    x = Dense(16, activation='relu',kernel_regularizer=regularizers.l2(0.0005))(x)    
    x = Dropout(0.1)(x)
    x = Dense(8, activation='relu',use_bias=False,kernel_regularizer=regularizers.l2(0.0005))(x)
    x = BatchNormalization()(x)
    predictions = Dense(1, activation='sigmoid',name="main")(x)
        
    model_created = Model(inputs=[input_embedding_1,input_embedding_2,meta_input], outputs=[predictions,emb_x_out])
    model_created.Name = name
    return model_created

In [22]:
def level_siames_merge_layer_soft(df_meta_input,df_embeddings_1,df_embeddings_2,name,bs):    
    
    input_embedding_1 = Input(shape=(len(df_embeddings_1.columns),), name='embedding_input_1')        
    input_embedding_2 = Input(shape=(len(df_embeddings_2.columns),), name='embedding_input_2')        
    
    emb_x_1 = get_tensor_embedding_layer(input_embedding_1)    
    emb_x_2 = get_tensor_embedding_layer(input_embedding_2)    
    
    L1_layer = Lambda(lambda tensors:K.abs(tensors[0] - tensors[1]))
    L1_distance = L1_layer([emb_x_1, emb_x_2])
    
    emb_x_out = Dense(3, activation='softmax',name="aux")(L1_distance)                
    
    meta_input = Input(shape=(len(df_meta_input.columns),), name='meta_input')    
    
    x = keras.layers.concatenate([L1_distance,meta_input])
    x = Dense(128, activation='relu',use_bias=False)(x)    
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    x = Dense(64, activation='relu',use_bias=False,kernel_regularizer=regularizers.l2(0.001))(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    x = Dense(32, activation='relu',use_bias=False,kernel_regularizer=regularizers.l2(0.001))(x)   
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    x = Dense(32, activation='relu',use_bias=False,kernel_regularizer=regularizers.l2(0.001))(x)    
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    x = Dense(16, activation='relu',kernel_regularizer=regularizers.l2(0.0005))(x)    
    x = Dropout(0.1)(x)
    x = Dense(8, activation='relu',use_bias=False,kernel_regularizer=regularizers.l2(0.0005))(x)
    x = BatchNormalization()(x)
    predictions = Dense(3, activation='softmax',name="main")(x)
        
    model_created = Model(inputs=[input_embedding_1,input_embedding_2,meta_input], outputs=[predictions,emb_x_out])
    model_created.Name = name
    return model_created


In [23]:
def euclidean_distance(x,y):    
    sum_square = K.sum(K.square(x - y), axis=1, keepdims=True)
    return K.sqrt(K.maximum(sum_square, K.epsilon()))


def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)



In [24]:
def fit_model_siames(train_x,train_emb_x_1,train_emb_x_2,train_y,val_x,val_emb_x_1,val_emb_x_2,val_y,model_train,n_epochs,optimizer,batchsize,loss_weigths,verb):
    tensorboard = TensorBoard(log_dir= train_filepath + working_level + "/board_logs/" + model_train.Name + "-{}".format(time()))
    checkpoint = ModelCheckpoint(train_filepath + working_level + "/model_checkpoints/{0}-check-{{epoch:02d}}-{{val_main_acc:.2f}}.hdf5".format(model_train.Name),save_weights_only=True,  period = int(n_epochs/5))
    best_model_save = ModelCheckpoint(train_filepath + working_level + "/model_checkpoints/{0}-best.hdf5".format(model_train.Name), monitor='val_main_acc', save_weights_only=True, save_best_only=True, mode='max')
    logger = EpochLogger(display=25)

    model_train.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy',km.binary_recall()],loss_weights=loss_weigths)    
    return model_train.fit([train_emb_x_1,train_emb_x_2,train_x], y = [train_y,train_y],verbose = verb,validation_data=([val_emb_x_1,val_emb_x_2,val_x],[val_y,val_y]),epochs = n_epochs,batch_size=batchsize,callbacks = [tensorboard,checkpoint,best_model_save,logger])  # starts training


def fit_model_siames_soft(train_x,train_emb_x_1,train_emb_x_2,train_y,val_x,val_emb_x_1,val_emb_x_2,val_y,model_train,n_epochs,optimizer,batchsize,loss_weigths,verb):
    tensorboard = TensorBoard(log_dir=train_filepath + working_level + "/board_logs/" + model_train.Name + "-{}".format(time()))
    checkpoint = ModelCheckpoint(train_filepath + working_level + "/model_checkpoints/{0}-check-{{epoch:02d}}-{{val_main_acc:.2f}}.hdf5".format(model_train.Name),save_weights_only=True,  period = int(n_epochs/5))
    best_model_save = ModelCheckpoint(train_filepath + working_level + "/model_checkpoints/{0}-best.hdf5".format(model_train.Name), monitor='val_main_acc', save_weights_only=True, save_best_only=True, mode='max')
    logger = EpochLogger(display=25)

    model_train.compile(optimizer=optimizer,loss='sparse_categorical_crossentropy',metrics=['accuracy',km.sparse_categorical_recall()],loss_weights=loss_weigths)    
    return model_train.fit([train_emb_x_1,train_emb_x_2,train_x], y = [train_y,train_y],verbose = verb,validation_data=([val_emb_x_1,val_emb_x_2,val_x],[val_y,val_y]),epochs = n_epochs,batch_size=batchsize,callbacks = [tensorboard,checkpoint,best_model_save,logger])  # starts training

In [34]:
species_siames_dict = {}
for specie in species_metadata:
    if(specie in ["Specie_Chicken"]):
        continue
    dataset_complete_no_species = dataset_complete[dataset_complete[specie] == 0]    
    dataset_complete_species = dataset_complete[dataset_complete[specie] == 1]    
    
    df_train,df_val,df_test = get_train_val_test(dataset_complete_no_species)
    
    df_train_x = df_train.drop(["Is_Ohnolog","Is_Paralog","Dup_Class"],axis=1)
    df_train_x = df_train_x.drop(embedding_1_cols,axis=1)
    df_train_x = df_train_x.drop(embedding_2_cols,axis=1)

    df_train_emb_x_1 = df_train[embedding_1_cols]
    df_train_emb_x_2 = df_train[embedding_2_cols]
    df_train_y = df_train["Is_Ohnolog"]

    df_val_x = df_val.drop(["Is_Ohnolog","Is_Paralog","Dup_Class"],axis=1)
    df_val_x = df_val_x.drop(embedding_1_cols,axis=1)
    df_val_x = df_val_x.drop(embedding_2_cols,axis=1)

    df_val_emb_x_1 = df_val[embedding_1_cols]
    df_val_emb_x_2 = df_val[embedding_2_cols]                  
    df_val_y = df_val["Is_Ohnolog"]

    df_test_x = df_test.drop(["Is_Ohnolog","Is_Paralog","Dup_Class"],axis=1)
    df_test_x = df_test_x.drop(embedding_1_cols,axis=1)
    df_test_x = df_test_x.drop(embedding_2_cols,axis=1)

    df_test_emb_x_1 =  df_test[embedding_1_cols]
    df_test_emb_x_2 =  df_test[embedding_2_cols]
    df_test_y = df_test["Is_Ohnolog"]
    
    model = level_siames_merge_layer(df_train_x,df_train_emb_x_1,df_train_emb_x_2,"Siames " + specie,128)
    
    log = fit_model_siames(df_train_x.values,df_train_emb_x_1.values,df_train_emb_x_2.values,df_train_y.values,df_val_x.values,df_val_emb_x_1.values,df_val_emb_x_2.values,df_val_y.values,model,800,Adamax(),256,[1,0.2],0)
    
    df_test_species_x = dataset_complete_species.drop(["Is_Ohnolog","Is_Paralog","Dup_Class"],axis=1)
    df_test_species_x = df_test_species_x.drop(embedding_1_cols,axis=1)
    df_test_species_x = df_test_species_x.drop(embedding_2_cols,axis=1)

    df_test_species_emb_x_1 = dataset_complete_species[embedding_1_cols]
    df_test_species_emb_x_2 = dataset_complete_species[embedding_2_cols]
    df_test_species_y = dataset_complete_species["Is_Ohnolog"]
    
    species_siames_dict[specie] = {}
    
    print("Evaluando especie Test: " + specie)
    metrics_names,metrics = (model.metrics_names,model.evaluate([df_test_emb_x_1.values,df_test_emb_x_2.values,df_test_x.values],[df_test_y.values,df_test_y.values], batch_size=256,verbose=False))
    count = 0
    for i in range(0,len(metrics)):
        print(f'{metrics_names[i]}     = {metrics[i]:.4f}')  
    print("\n")
    
    species_siames_dict[specie]["Test"] = metrics[5]
            
        
    print("Evaluando especie sin balancear: " + specie)
    metrics_names,metrics = (model.metrics_names,model.evaluate([df_test_species_emb_x_1.values,df_test_species_emb_x_2.values,df_test_species_x.values],[df_test_species_y.values,df_test_species_y.values], batch_size=256,verbose=False))
    count = 0
    for i in range(0,len(metrics)):
        print(f'{metrics_names[i]}     = {metrics[i]:.4f}')  
    print("\n")
    
    species_siames_dict[specie]["Sin Balanceo"] = metrics[5]
    
    df_species_ohnologs = dataset_complete_species[dataset_complete_species["Is_Ohnolog"] == 1]
    df_species_no_ohnologs = dataset_complete_species[dataset_complete_species["Is_Ohnolog"] == 0]
    
    if(len(df_species_ohnologs)>len(df_species_no_ohnologs)):
        df_species_new = df_species_no_ohnologs.append(df_species_ohnologs.sample(len(df_species_no_ohnologs)))        
    else:
        df_species_new = df_species_ohnologs.append(df_species_no_ohnologs.sample(len(df_species_ohnologs)))

    if(len(df_species_new) == 0):
        df_species_new = df_species_ohnologs        
                
    df_test_species_x = df_species_new.drop(["Is_Ohnolog","Is_Paralog","Dup_Class"],axis=1)
    df_test_species_x = df_test_species_x.drop(embedding_1_cols,axis=1)
    df_test_species_x = df_test_species_x.drop(embedding_2_cols,axis=1)

    df_test_species_emb_x_1 =  df_species_new[embedding_1_cols]
    df_test_species_emb_x_2 =  df_species_new[embedding_2_cols]
    df_test_species_y = df_species_new["Is_Ohnolog"]    
    
    print("Evaluando especie balanceada: " + specie)
    metrics_names,metrics = (model.metrics_names,model.evaluate([df_test_species_emb_x_1.values,df_test_species_emb_x_2.values,df_test_species_x.values],[df_test_species_y.values,df_test_species_y.values], batch_size=256,verbose=False))
    count = 0
    for i in range(0,len(metrics)):
        print(f'{metrics_names[i]}     = {metrics[i]:.4f}')  
    print("\n")
    
    species_siames_dict[specie]["Balanceadas"] = metrics[5]   

step: 25/800 ...  - loss: 4.2965 - val_loss: 4.0877
 - main_loss: 0.5136 - main_acc: 0.7410 - main_recall: 0.5639 - val_main_loss: 0.4786 - val_main_acc: 0.7689 - val_main_recall: 0.5765
 - aux_loss: 0.5890 - aux_acc: 0.6669 - aux_recall: 0.5632 - val_aux_loss: 0.5845 - val_aux_acc: 0.6695 - val_aux_recall: 0.5713
step: 50/800 ...  - loss: 0.9584 - val_loss: 0.9528
 - main_loss: 0.4195 - main_acc: 0.8115 - main_recall: 0.6217 - val_main_loss: 0.4222 - val_main_acc: 0.8152 - val_main_recall: 0.6179
 - aux_loss: 0.5711 - aux_acc: 0.6730 - aux_recall: 0.6196 - val_aux_loss: 0.5898 - val_aux_acc: 0.6606 - val_aux_recall: 0.6070
step: 75/800 ...  - loss: 0.7209 - val_loss: 0.7265
 - main_loss: 0.3971 - main_acc: 0.8238 - main_recall: 0.6260 - val_main_loss: 0.4012 - val_main_acc: 0.8254 - val_main_recall: 0.6156
 - aux_loss: 0.5698 - aux_acc: 0.6722 - aux_recall: 0.6241 - val_aux_loss: 0.5871 - val_aux_acc: 0.6634 - val_aux_recall: 0.6039
step: 100/800 ...  - loss: 0.6638 - val_loss: 0.6736

In [38]:
species_siames_cross_dict = {}
for specie in species_metadata:
    dataset_complete_no_species = dataset_complete[dataset_complete[specie] == 0]    
    dataset_complete_species = dataset_complete[dataset_complete[specie] == 1]    
        
    df_no_species = dataset_complete_no_species[dataset_complete_no_species[specie] == 0]
    
    df_train, df_val = train_test_split(df_no_species, test_size=0.2,random_state=9,stratify=df_no_species["Is_Ohnolog"])
    
    df_train_x = df_train.drop(["Is_Ohnolog","Is_Paralog","Dup_Class"],axis=1)
    df_train_x = df_train_x.drop(embedding_1_cols,axis=1)
    df_train_x = df_train_x.drop(embedding_2_cols,axis=1)
    
    df_train_emb_x_1 = df_train[embedding_1_cols]
    df_train_emb_x_2 = df_train[embedding_2_cols]

    model = level_siames_merge_layer(df_train_x,df_train_emb_x_1,df_train_emb_x_2,"Siames " + specie,128)        
    specie_metadata_copy = species_metadata[:]
    specie_metadata_copy.remove(specie)

    for specie_cross in specie_metadata_copy:
        df_no_species = dataset_complete_no_species[dataset_complete_no_species[specie_cross] == 0]
        df_train, df_val = train_test_split(df_no_species, test_size=0.2,random_state=9,stratify=df_no_species["Is_Ohnolog"])

        df_train_x = df_train.drop(["Is_Ohnolog","Is_Paralog","Dup_Class"],axis=1)
        df_train_x = df_train_x.drop(embedding_1_cols,axis=1)
        df_train_x = df_train_x.drop(embedding_2_cols,axis=1)

        df_train_emb_x_1 = df_train[embedding_1_cols]
        df_train_emb_x_2 = df_train[embedding_2_cols]
        df_train_y = df_train["Is_Ohnolog"]

        df_val_x = df_val.drop(["Is_Ohnolog","Is_Paralog","Dup_Class"],axis=1)
        df_val_x = df_val_x.drop(embedding_1_cols,axis=1)
        df_val_x = df_val_x.drop(embedding_2_cols,axis=1)

        df_val_emb_x_1 = df_val[embedding_1_cols]
        df_val_emb_x_2 = df_val[embedding_2_cols]
        df_val_y = df_val["Is_Ohnolog"]

        print("Entrenando sin especie: " + specie)
        log = fit_model_siames(df_train_x.values,df_train_emb_x_1.values,df_train_emb_x_2.values,df_train_y.values,df_val_x.values,df_val_emb_x_1.values,df_val_emb_x_2.values,df_val_y.values,model,200,Adamax(),256,[1,0.2],0)
        
    
    df_test_species_x = dataset_complete_species.drop(["Is_Ohnolog","Is_Paralog","Dup_Class"],axis=1)
    df_test_species_x = df_test_species_x.drop(embedding_1_cols,axis=1)
    df_test_species_x = df_test_species_x.drop(embedding_2_cols,axis=1)

    df_test_species_emb_x_1 =  dataset_complete_species[embedding_1_cols]
    df_test_species_emb_x_2 =  dataset_complete_species[embedding_2_cols]
    df_test_species_y = dataset_complete_species["Is_Ohnolog"]
    
    species_siames_cross_dict[specie] = {}
    
    print("Evaluando especie sin balancear: " + specie)
    metrics_names,metrics = (model.metrics_names,model.evaluate([df_test_species_emb_x_1.values,df_test_species_emb_x_2.values,df_test_species_x.values],[df_test_species_y.values,df_test_species_y.values], batch_size=256,verbose=False))
    count = 0
    for i in range(0,len(metrics)):
        print(f'{metrics_names[i]}     = {metrics[i]:.4f}')  
    print("\n")
    
    species_siames_cross_dict[specie]["Sin Balanceo"] = metrics[5]
    
    df_species_ohnologs = dataset_complete_species[dataset_complete_species["Is_Ohnolog"] == 1]
    df_species_no_ohnologs = dataset_complete_species[dataset_complete_species["Is_Ohnolog"] == 0]
    
    if(len(df_species_ohnologs)>len(df_species_no_ohnologs)):
        df_species_new = df_species_no_ohnologs.append(df_species_ohnologs.sample(len(df_species_no_ohnologs)))        
    else:
        df_species_new = df_species_ohnologs.append(df_species_no_ohnologs.sample(len(df_species_ohnologs)))

    if(len(df_species_new) == 0):
        df_species_new = df_species_ohnologs        
        
    df_test_species_x = df_species_new.drop(["Is_Ohnolog","Is_Paralog","Dup_Class"],axis=1)
    df_test_species_x = df_test_species_x.drop(embedding_1_cols,axis=1)
    df_test_species_x = df_test_species_x.drop(embedding_2_cols,axis=1)

    df_test_species_emb_x_1 = df_species_new[embedding_1_cols]
    df_test_species_emb_x_2 = df_species_new[embedding_2_cols]
    df_test_species_y = df_species_new["Is_Ohnolog"]                    
    
    print("Evaluando especie balanceada: " + specie)
    metrics_names,metrics = (model.metrics_names,model.evaluate([df_test_species_emb_x_1.values,df_test_species_emb_x_2.values,df_test_species_x.values],[df_test_species_y.values,df_test_species_y.values], batch_size=256,verbose=False))
    count = 0
    for i in range(0,len(metrics)):
        print(f'{metrics_names[i]}     = {metrics[i]:.4f}')  
    print("\n")
    
    species_cross_dict[specie]["Balanceadas"] = metrics[5]   

Entrenando sin especie: Specie_Chicken
step: 25/200 ...  - loss: 3.0230 - val_loss: 2.8215
 - main_loss: 0.4849 - main_acc: 0.7591 - main_recall: 0.5942 - val_main_loss: 0.4606 - val_main_acc: 0.7790 - val_main_recall: 0.5991
 - aux_loss: 0.5757 - aux_acc: 0.6693 - aux_recall: 0.5927 - val_aux_loss: 0.5753 - val_aux_acc: 0.6817 - val_aux_recall: 0.5952
step: 50/200 ...  - loss: 0.7976 - val_loss: 0.8003
 - main_loss: 0.4130 - main_acc: 0.8150 - main_recall: 0.6312 - val_main_loss: 0.4161 - val_main_acc: 0.8141 - val_main_recall: 0.6274
 - aux_loss: 0.5661 - aux_acc: 0.6744 - aux_recall: 0.6292 - val_aux_loss: 0.5768 - val_aux_acc: 0.6804 - val_aux_recall: 0.6226
step: 75/200 ...  - loss: 0.6785 - val_loss: 0.6850
 - main_loss: 0.3902 - main_acc: 0.8278 - main_recall: 0.6468 - val_main_loss: 0.3941 - val_main_acc: 0.8256 - val_main_recall: 0.6499
 - aux_loss: 0.5604 - aux_acc: 0.6762 - aux_recall: 0.6448 - val_aux_loss: 0.5747 - val_aux_acc: 0.6687 - val_aux_recall: 0.6455
step: 100/200